In [1]:

import os
import warnings
import sys
sys.path.append('../')

import torch
import torch.nn as nn

import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from resnet import *

from main import *

model_names = sorted(name for name in models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(models.__dict__[name]))

In [2]:
# args = parser.parse_args(args=[])
# args = parser.parse_args()
import easydict 
args = easydict.EasyDict({ "batch-size": 256, 
                          "epochs": 100,
                          "data": 0, 
                          'arch':'resnet18',
                          'lr':0.1,
                         'momentum':0.9,
                         'weight_decay':1e-4,
                         'start_epoch':0,
                         'gpu':0,
                         'saved_dir':'../trained_model/model_best.pt'})


In [3]:
# device = torch.device('cpu')
device = torch.device('cuda')
print(device)

cuda


In [4]:
print("=> using pre-trained model '{}'".format('resnet18'))
# model = models.__dict__['resnet18'](pretrained=True)
# model = models.resnet18(pretrained=False)
model = resnet18(pretrained=False)
# model.to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), args.lr,
                            momentum=args.momentum,
                            weight_decay=args.weight_decay)

=> using pre-trained model 'resnet18'


In [5]:
saved_model = '../trained_model/model_best_afterRelu.pt'
# saved_model = '../trained_model/model_best_beforeRelu.pt'
state = torch.load(saved_model, map_location=torch.device('cpu'))
state_dict = state['state_dict']

In [6]:
import re
model_state_dict = {}
for i in state_dict :
    if 'module.' in i :
        s = re.sub('module.', '', i)
        model_state_dict[s] = state_dict[i]

In [7]:
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [8]:
# model.eval()

In [9]:
# Data loading code
data_dir = '../ILSVRC/Data/CLS-LOC/'
traindir = os.path.join(data_dir, 'train')
valdir = os.path.join(data_dir, 'val')

In [10]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
val_dataset = datasets.ImageFolder(valdir, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ]))

In [11]:
# next(iter(train_dataset))

In [12]:
# train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
train_sampler = None

In [13]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=256, shuffle=(train_sampler is None),
    num_workers=8, pin_memory=True, sampler=train_sampler)

In [14]:
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=32, shuffle=False,
    num_workers=8, pin_memory=True)

In [15]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (gelu): GELU()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (Noise): AddNoise()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu)

In [16]:
acc1 = validate(val_loader, model, criterion, args) 
print(acc1)

/home/beomgon/anaconda3/envs/pytorch_retina/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


 * Acc@1 70.536 Acc@5 89.535
tensor(70.5359, device='cuda:0')


In [17]:
# saved_model = '../trained_model/model_best_afterRelu.pt'
saved_model = '../trained_model/model_best_beforeRelu.pt'
state = torch.load(saved_model, map_location=torch.device('cpu'))
state_dict = state['state_dict']

import re
model_state_dict = {}
for i in state_dict :
    if 'module.' in i :
        s = re.sub('module.', '', i)
        model_state_dict[s] = state_dict[i]
        
model.load_state_dict(model_state_dict)        

<All keys matched successfully>

In [18]:
model.to(device)
acc1 = validate(val_loader, model, criterion, args) 
print(acc1)

 * Acc@1 70.592 Acc@5 89.597
tensor(70.5919, device='cuda:0')
